In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# Overview of dataset



The data was obtained from [the Broad Institute](http://portals.broadinstitute.org/cgi-bin/cancer/publications/view/43) and is stored as follows:

<table border="2" cellspacing="0" cellpadding="6" rules="groups" frame="hsides">


<colgroup>
<col  class="org-left" />

<col  class="org-left" />
</colgroup>
<thead>
<tr>
<th scope="col" class="org-left">Type of data</th>
<th scope="col" class="org-left">File name</th>
</tr>
</thead>

<tbody>
<tr>
<td class="org-left">Training data</td>
<td class="org-left">`data_set_ALL_AML_train.txt`</td>
</tr>


<tr>
<td class="org-left">Training data class labels</td>
<td class="org-left">`ALL_vs_AML_train_set_38_sorted.cls`</td>
</tr>


<tr>
<td class="org-left">Testing data</td>
<td class="org-left">`data_set_ALL_AML_independent.txt`</td>
</tr>


<tr>
<td class="org-left">Testing data class labels</td>
<td class="org-left">`Leuk_ALL_AML.test.cls`</td>
</tr>
</tbody>
</table>




# Cleaning the data



In [1]:
def clean_training_data():
    clean_lines = []
    with open("data_set_ALL_AML_train.txt", "r") as f:
        lines = f.readlines()
        clean_lines = [l.rstrip('\t\n') for l in lines]

    with open("data_set_ALL_AML_train_cleaned.txt", "w") as f:
        f.writelines('\n'.join(clean_lines))

clean_training_data()


# Loading the data



In [2]:
import numpy, scipy, pandas
import sklearn
import re

def load_data(x_filename, y_filename):
    df_x = pandas.read_csv(x_filename, sep="\t")
    df_x = df_x.select(lambda x: not re.search('call\.*', x), axis=1)
    df_x = df_x.drop(['Gene Description', 
                      'Gene Accession Number'], axis=1)
    df_x = df_x.T
    x = df_x.values

    with open(y_filename, "r") as fin:
        data = fin.read().splitlines(True)
    data = data[1].rstrip()

    y = numpy.fromstring(data, sep=" ")

    return x, y


x_train, y_train = load_data("data_set_ALL_AML_train_cleaned.txt",
                             "ALL_vs_AML_train_set_38_sorted.cls")
x_test, y_test = load_data("data_set_ALL_AML_independent.txt",
                           "Leuk_ALL_AML.test.cls")
y_test = y_test[1:]  # dataset has one additional 0 at beginning, 
                     # so remove it


# Run models




## Kernelized logistic regression with L2 and L1 regularization, logistic regression with L1 regularization



To choose the $\gamma$ function of the RBF kernel (where $\gamma = 1/(2\sigma^2)$) we follow the heuristic choice mentioned in Gretton et al. (p. 748) of setting $\sigma$ to equal the median distance between points of the training data.



In [3]:
import sklearn.linear_model
import sklearn.kernel_ridge
import sklearn.metrics.pairwise
from sklearn.metrics import confusion_matrix
from scipy.spatial.distance import cdist
import statistics

#import tabulate

# Calculate gamma as in Gretton et al.
b = cdist(x_train, x_train).ravel()
gamma = 1/(2 * pow(statistics.median(b), 2))

#y_test_onehot = numpy.zeros((len(y_test), 2))
#y_test_onehot[numpy.arange(len(y_test)), y_test.astype(int)] = 1

# Calculate RBF kernel 
K      = sklearn.metrics.pairwise.rbf_kernel(x_train, x_train, gamma=gamma)
K_test = sklearn.metrics.pairwise.rbf_kernel(x_test, x_train, gamma=gamma)

# Fit kernelized logistic regression
# (note that l2 regularization is applied by default)
clf = sklearn.linear_model.LogisticRegression(solver='lbfgs')
clf.fit(K, y_train)
kernelized_l2_preds = clf.predict(K_test)

# Fit kernelized logistic regression with l1 regularization
# (note that liblinear solver used by default)
clf = sklearn.linear_model.LogisticRegression(penalty='l1')
clf.fit(K, y_train)
kernelized_l1_preds = clf.predict(K_test)

# Fit non-kernelized logistic regression with l1 regularization
clf = sklearn.linear_model.LogisticRegression(penalty='l1')
clf.fit(x_train, y_train)
l1_preds = clf.predict(x_test)


## Evaluation of results



In [9]:
from sklearn.metrics import zero_one_loss

kernelized_l2_er = zero_one_loss(y_test, kernelized_l2_preds)
kernelized_l1_er = zero_one_loss(y_test, kernelized_l1_preds)
l1_er = zero_one_loss(y_test, l1_preds)

kernelized_l2_cm = confusion_matrix(y_test, kernelized_l2_preds)
kernelized_l1_cm = confusion_matrix(y_test, kernelized_l1_preds)
l1_cm = confusion_matrix(y_test, l1_preds)


## SVM



In [12]:
from sklearn.svm import SVC
#from sklearn.cross_validation import StratifiedKFold
#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

# This code is a modification of code at
# http://ogrisel.github.io/scikit-learn.org/sklearn-tutorial/auto_examples/svm/plot_svm_parameters_selection.html

def find_svm_best_params(kernel_type):
    C_range = 2. ** numpy.arange(-5, 15, 2)
    gamma_range = 2. ** numpy.arange(-5, 3, 2)

    param_grid = dict(gamma=gamma_range, C=C_range)

    grid_search = GridSearchCV(SVC(kernel=kernel_type), param_grid, cv=5)
    grid_search.fit(x_train, y_train)
    bestparams = grid_search.best_params_
    #results_dict = grid_search.cv_results_
    #for param, score_mean, score_sd in zip(results_dict['params'],        
    #results_dict['mean_test_score'], results_dict['std_test_score']):
    #    print(param, round(score_mean, 4), round(score_sd, 4))
    return bestparams

def estimate_svm(C, gamma, kernel_type, x_train, y_train, x_test, y_test):
    our_svm = SVC(kernel=kernel_type, C=C, gamma=gamma)
    our_svm.fit(x_train, y_train)
    svm_preds = our_svm.predict(x_test)
    svm_er = zero_one_loss(y_test, svm_preds)
    svm_cm = confusion_matrix(y_test, svm_preds)
    return svm_er, svm_cm

bestparams_rbf = find_svm_best_params("rbf")
svm_rbf_er, svm_rbf_cm = estimate_svm(bestparams_rbf['C'],
                                      bestparams_rbf['gamma'],
                                      "rbf",
                                      x_train, y_train, x_test, y_test)
bestparams_linear = find_svm_best_params("linear")
svm_linear_er, svm_linear_cm = estimate_svm(bestparams_linear['C'], 
                                            bestparams_linear['gamma'],
                                            "linear",
                                            x_train, y_train, x_test, y_test)


## Print results



In [11]:
from IPython.display import HTML, display
import pandas

results_df = pandas.DataFrame(columns=('Model', 'Empirical error'))

series_index = ["Model", "Empirical error"]
results_df = results_df.append(pandas.Series(["Kernelized L2", 
                                             kernelized_l2_er], 
                                index=series_index),
                  ignore_index=True)

results_df = results_df.append(pandas.Series(["Kernelized L1", 
                                              kernelized_l1_er], 
                                index=series_index),
                  ignore_index=True)

results_df = results_df.append(pandas.Series(["L1", l1_er], 
                                index=series_index),
                  ignore_index=True)

results_df = results_df.append(pandas.Series(["SVM (RBF kernel)", 
                                              svm_rbf_er], 
                                             index=series_index),
                  ignore_index=True)
results_df = results_df.append(pandas.Series(["SVM (Linear kernel)", 
                                              svm_linear_er], 
                                             index=series_index),
                  ignore_index=True)

#display(HTML(results_df.to_html()))

display(results_df)

,Model,Empirical error
0,Kernelized L2,0.264706
1,Kernelized L1,0.411765
2,L1,0.000000
3,SVM (RBF kernel),0.411765
4,SVM (Linear kernel),0.029412



# References (move to separate file later)



Gretton, Arthur et al. 2012. "A Kernel Two-Sample Test." *Journal of Machine Learning Research*. Vol 13, p. 723-773.

Hsu, Chih-Wei et al. 2016. "A Practical Guide to Support Vector Classification." Department of Computer Science, National Taiwan University.

